In [2]:
import numpy as np
import cv2
from skimage import exposure, morphology, segmentation
import pandas as pd
import scipy.sparse as sparse
import os

In [4]:
import sys
sys.path.insert(0,'/h/resolve_glioblastoma/data/')
from ResolveTools.image.utils import save_tiff, claher, read_single_modality_confocal, save_tiff_from_float, resize_shrink
from ResolveTools.resolve.resolveimage import ResolveImage

In [3]:
def generate_zDistribution(name):
    rim = ResolveImage("01_raw/T6GBM_"+name+"_transcripts.txt")
    rim.add_metadata("../metadata/gbm_resolve_genes.xlsx")
    
    rim.full_data["x"] = rim.full_data["x"]//8
    rim.full_data["y"] = rim.full_data["y"]//8
    rim.full_data["z"] = rim.full_data["z"]//2

    rim.full_data["x"] = rim.full_data["x"] - rim.full_data["x"].min()
    rim.full_data["y"] = rim.full_data["y"] - rim.full_data["y"].min()
    rim.full_data["z"] = rim.full_data["z"] - rim.full_data["z"].min()

    img = np.zeros([rim.full_data["z"].max()+1, 3, rim.full_data["x"].max()+1, rim.full_data["y"].max()+1])

    def add_df(img, df, i):
        img[df["z"],i,df["x"],df["y"]] = 1
    
    for gene in rim.genes[rim.genes["Species"]=="Mouse"]["GeneR"]:
        add_df(img, rim.full_data[rim.full_data["GeneR"]==gene], 0)

    for gene in rim.genes[rim.genes["Species"]=="Human"]["GeneR"]:
        add_df(img, rim.full_data[rim.full_data["GeneR"]==gene], 2 if gene == "MCHERRY" else 1)

    img = (img*255).astype("H")
    save_tiff("02_QC/T6GBM_"+name+"_zDistribution.tif", img)

In [9]:
names = np.unique([l[6:13] for l in os.listdir("01_raw")])
for name in names:
    print(name)
    if "R1" not in name: generate_zDistribution(name)

['R1_W0A1' 'R1_W0A2' 'R1_W1A1' 'R1_W2A1' 'R1_W3A1' 'R1_W3A2' 'R1_W3A3'
 'R1_W5A2' 'R1_W5A3' 'R1_W6A1' 'R1_W6A2' 'R1_W7A1' 'R1_W7A2' 'R2_W0A1'
 'R2_W0A2' 'R2_W1A1' 'R2_W2A1' 'R2_W3A1' 'R2_W4A1' 'R2_W5A1' 'R2_W5A2'
 'R2_W6A1' 'R2_W7A1']
R1_W0A1
R1_W0A2
R1_W1A1
R1_W2A1
R1_W3A1
R1_W3A2
R1_W3A3
R1_W5A2
R1_W5A3
R1_W6A1
R1_W6A2
R1_W7A1
R1_W7A2
R2_W0A1
R2_W0A2
R2_W1A1
R2_W2A1
R2_W3A1
R2_W4A1
R2_W5A1
R2_W5A2
R2_W6A1
R2_W7A1


In [47]:
def get_summary(name):
    rim = ResolveImage("01_raw/T6GBM_"+name+"_transcripts.txt")
    rim.add_metadata("../metadata/gbm_resolve_genes.xlsx")
    
    ct = rim.genes[["Count"]].copy()
    ct.columns = [name]
    
    vol = np.prod(rim.full_data[["x","y"]].max(axis=0))*np.prod(rim.voxelsize[:2])
    print(name+":",np.round(float(ct.sum()/vol)*100,2),"counts per 100 um^2.")
    return ct

In [48]:
names = np.unique([l[6:13] for l in os.listdir("01_raw")])
cts = [get_summary(name) for name in names]

R1_W0A1: 7.93 counts per 100 um^2.
R1_W0A2: 10.39 counts per 100 um^2.
R1_W1A1: 6.94 counts per 100 um^2.
R1_W2A1: 7.57 counts per 100 um^2.
R1_W3A1: 7.96 counts per 100 um^2.
R1_W3A2: 8.93 counts per 100 um^2.
R1_W3A3: 6.13 counts per 100 um^2.
R1_W5A2: 5.97 counts per 100 um^2.
R1_W5A3: 3.47 counts per 100 um^2.
R1_W6A1: 2.85 counts per 100 um^2.
R1_W6A2: 6.18 counts per 100 um^2.
R1_W7A1: 14.39 counts per 100 um^2.
R1_W7A2: 7.6 counts per 100 um^2.
R2_W0A1: 15.74 counts per 100 um^2.
R2_W0A2: 28.19 counts per 100 um^2.
R2_W1A1: 15.21 counts per 100 um^2.
R2_W2A1: 25.62 counts per 100 um^2.
R2_W3A1: 13.7 counts per 100 um^2.
R2_W4A1: 9.47 counts per 100 um^2.
R2_W5A1: 3.96 counts per 100 um^2.
R2_W5A2: 7.76 counts per 100 um^2.
R2_W6A1: 7.88 counts per 100 um^2.
R2_W7A1: 6.55 counts per 100 um^2.


In [49]:
df = pd.concat(cts, axis=1)
df[df!=df] = 0
df.to_csv("02_QC/occurences.csv", sep="\t")

In [51]:
df.loc["MCHERRY"]

R1_W0A1     3332
R1_W0A2    10570
R1_W1A1     8441
R1_W2A1    12126
R1_W3A1     9633
R1_W3A2     1470
R1_W3A3     3372
R1_W5A2        1
R1_W5A3        1
R1_W6A1        2
R1_W6A2        0
R1_W7A1        0
R1_W7A2        0
R2_W0A1     1243
R2_W0A2      207
R2_W1A1     2133
R2_W2A1     1141
R2_W3A1      999
R2_W4A1        5
R2_W5A1        1
R2_W5A2        1
R2_W6A1    11884
R2_W7A1     8400
Name: MCHERRY, dtype: object

# Special Gene Distribution

In [10]:
dapi = read_single_modality_confocal("../confocal/R1_01_image/Confocal_R1_W3A1_DAPI.tif")
dapi = np.asarray([resize_shrink(img, 3) for img in dapi])[:,None]
mcherry = read_single_modality_confocal("../confocal/R1_01_image/Confocal_R1_W3A1_mCherry.tif")
mcherry = np.asarray([resize_shrink(img, 3) for img in mcherry])[:,None]

In [11]:
dapi.shape

(17, 1, 2890, 4235)

In [16]:
rim = ResolveImage("../resolve/R1_04_registration3D/T6GBM_R1_W3A1_transcripts_registered3D.txt")
rim.add_metadata("../metadata/gbm_resolve_genes.xlsx")
rim.full_data[["x","y"]] = rim.full_data[["x","y"]]//3

In [19]:
genesD = set(rim.genes[rim.genes["Celltype"]=="Differentiation"]["GeneR"])
genesHuman_woD = set(rim.genes[rim.genes["Species"]!="Mouse"]["GeneR"]) - genesD
genesMouse = set(rim.genes[rim.genes["Species"]=="Mouse"]["GeneR"])

In [34]:
rim.full_data["color"] = 2
rim.full_data.loc[rim.full_data["GeneR"].apply(lambda x: x in genesHuman_woD), "color"] = 1
rim.full_data.loc[rim.full_data["GeneR"].apply(lambda x: x in genesD), "color"] = 0

In [38]:
shape = [dapi.shape[0]]+[3]+list(dapi.shape[-2:])

In [40]:
counts = np.zeros(shape)

In [49]:
list(range(-1,2))

[-1, 0, 1]

In [56]:
(rim.full_data["z"]-1).min(axis)

2

In [57]:
for i in range(2):
    for j in range(2):
        for k in range(-1,2):
            counts[np.clip(rim.full_data["z"]+k, 0, counts.shape[0]-1),
                   rim.full_data["color"], rim.full_data["y"]+i, rim.full_data["x"]+j] = 1

In [ ]:
(counts!=0).sum()

In [58]:
image = np.concatenate([dapi, mcherry, counts], axis=1)

In [59]:
save_tiff_from_float("02_QC/T6GBM_R1_W3A1_Dtest.tif", image)

# Appendix

In [53]:
name = "W3A1"
rim = ResolveImage("R1_01_raw/T6GBM_R1_"+name+"_transcripts.txt",
                  {#"nuclei_resolve": "R1_01_raw/T6GBM_R1_"+name+"_DAPI.tiff",
                   })
genes = pd.read_excel("../metadata/gbm_resolve_genes.xlsx").fillna("")
genes.index = [gene.upper() if sp!="Mouse" else gene.upper()+"_M" for gene, sp in zip(genes["Gene"], genes["Species"])]
#genes["GeneMod"] = [gene.upper() if sp!="Mouse" else gene.upper()+"_M" for gene, sp in zip(genes["Gene"], genes["Species"])]
#genes.index = np.asarray(genes["GeneMod"])
rim.genes = pd.merge(rim.genes,genes,left_index=True,right_index=True,how="left").sort_values("Count",ascending=False)

In [54]:
rim.full_data["x"] = rim.full_data["x"]//8
rim.full_data["y"] = rim.full_data["y"]//8
rim.full_data["z"] = rim.full_data["z"]//2

In [55]:
rim.full_data["x"] = rim.full_data["x"] - rim.full_data["x"].min()
rim.full_data["y"] = rim.full_data["y"] - rim.full_data["y"].min()
rim.full_data["z"] = rim.full_data["z"] - rim.full_data["z"].min()

In [56]:
img = np.zeros([rim.full_data["z"].max()+1, 3, rim.full_data["x"].max()+1, rim.full_data["y"].max()+1])

In [57]:
def add_df(img, df, i):
    img[df["z"],i,df["x"],df["y"]] = 1

In [58]:
for gene in rim.genes[rim.genes["Species"]=="Mouse"]["GeneR"]:
    add_df(img, rim.full_data[rim.full_data["GeneR"]==gene], 0)

In [59]:
for gene in rim.genes[rim.genes["Species"]=="Human"]["GeneR"]:
    add_df(img, rim.full_data[rim.full_data["GeneR"]==gene], 2 if gene == "MCHERRY" else 1)

In [60]:
img = (img*255).astype("H")

In [63]:
save_tiff("R1_02_QC/T6GBM_R1_"+name+"_zDistribution.tif", img)